# 모델 구현하기

In [2]:
import tensorflow as tf

# MNIST 데이터셋 사용
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data", one_hot=True)

W0725 15:49:24.778151 4386469312 deprecation.py:323] From <ipython-input-2-0eedb7f90c67>:5: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0725 15:49:24.779016 4386469312 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0725 15:49:24.779839 4386469312 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz


W0725 15:49:25.000919 4386469312 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


MNIST의 레이블은 0~9의 값이지만, 이것은 연속된 숫자가 아닌 카테고리 값입니다. 따라서 One-Hot Encoding이 필요합니다. 
* **ex)** 5는 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], 0은 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0] 식으로 됩니다.

---

* **X: 입력 데이터를 넣을 플레이스 홀더**
  * None : 입력 데이터의 개수 (텐서플로가 알아서 계산할 수 있도록 미지정을 해놓음)
  * 28, 28 : 28 X 28 이미지를 뜻한다.
  * 1 : 이미지가 회색조이기 때문에 채널에 색상이 하나이다.

<br>

* **Y: 출력 데이터가 나올 플레이스 홀더**
  * None : 출력 데이터의 개수 (텐서플로가 알아서 계산할 수 있도록 미지정을 해놓음)
  * 10 : 숫자 이미지 파일이 0 부터 10 까지 있으므로 결과는 10개 중에 하나이다.

In [3]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

## CNN 계층 구성하기
* **3 X 3 크기의 커널을 가진 컨볼루션 계층을 만든다.**
  1. 3 X 3 커널이 오른쪽과 아래쪽으로 한 칸씩(스트라이드) 움직이는 32개의 커널을 가진 컨볼루션 계층을 만들기 위한 가중치를 만든다.
  2. tf.nn.conv2d 함수를 이용하여 컨볼루션 계층을 만든다. (padding='SAME': 이미지의 테두리의 한 칸 밖까지 움직이는 옵션이다. 이렇게 하면 이미지의 테두리 까지 명확한 평가가 가능)
  3. tf.nn.relu 활성화 함수를 통해 컨볼루션 계층을 완성한다.

In [4]:
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)

## 폴링 계층 구성하기
* 컨볼루션 계층을 입력층으로 사용하고, 커널 크기를 2 X 2로 하는 풀링 계층을 만든다.

In [5]:
# strides=[1, 2, 2, 1] : 슬라이딩 시 두 칸씩 움직이겠다는 옵션이다.
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                   padding='SAME')

## 첫 번째 CNN 계층 구성
<img src="../capture/스크린샷 2019-07-24 오후 11.10.36.png">

## 두 번째 CNN 계층 구성
* 3 X 3 크기의 커널 64개로 구성한 컨볼루션 계층과 2 X 2 크기의 폴링 계층으로 구성한다.
  * **W2(가중치) 변수의 구성**
    * 32 : 첫 번째 컨볼루션 계층의 커널 개수. 이것은 출력층의 개수이며 또한 첫 번째 컨볼루션 계층이 찾아낸 이미지 특징 개수라고 할 수 있다.

<img src="../capture/스크린샷 2019-07-24 오후 11.18.16.png">

In [6]:
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                   padding='SAME')

## 추출한 특징들을 이용해 10개의 분류를 만들어내는 계층 구성
1. **10개의 분류는 1차원 배열이므로 차원을 줄이는 단계가 필요하다.**
  * 위의 L2 폴링 계층 크기가 7 x 7 x 64 이므로 reshape 함수를 통해서 7 x 64 크기의 1차원 계층으로 만든다.
2. **위에서 만든 배열 전체를 최종 출력값의 중간 단계인 256개의 뉴런으로 연결하는 신경망을 만든다.**
> 인접한 계층의 모든 뉴런과 상호 연결된 계층을 **완전 연결 계층** 이라고 한다.
3. **추가로 과적합을 막아주는 드롭아웃 기법을 사용한다.**

In [7]:
W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

W0725 15:49:49.000101 4386469312 deprecation.py:506] From <ipython-input-7-49010747b473>:5: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## 모델 구성의 마지막
직전의 은닉층인 L3의 출력값 256개를 받아 최종 출력값인 0~9 레이블을 갖는 10개의 출력값을 만든다.

In [8]:
W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

reduce_mean을 통해 **손실 함수**를 만들고, AdamOptimizer를 이용하여 **최적화 함수**를 만든다.

In [9]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

## 학습과 결과 확인
1. 신경망 모델 초기화하고 학습 진행할 세션 시작
2. 미니 배치의 크기 100으로 설정
3. 학습 횟수를 지정하고 미니 배치의 총 개수만큼 반복 학습할 수 있도록한다.
4. 이미지 데이터를 28 x 28로 재구성해준다.
5. 최적화시키고 손실값 저장
6. 예측 결과인 모델과 실제 값을 비교한다.

In [10]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

for epoch in range(15):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost],
                              feed_dict={X: batch_xs,
                                        Y: batch_ys,
                                        keep_prob: 0.7})
        total_cost += cost_val
        
    print('Epoch: ', '%4d' %(epoch + 1),
         'Avg. cost = ', '{:.3f}'.format(total_cost / total_batch))

print('최적화 완료')

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: ', sess.run(accuracy,
                       feed_dict={X: mnist.test.images.reshape(
                                                 -1, 28, 28, 1),
                                 Y: mnist.test.labels,
                                 keep_prob: 1}))

Epoch:     1 Avg. cost =  0.342
Epoch:     2 Avg. cost =  0.109
Epoch:     3 Avg. cost =  0.078
Epoch:     4 Avg. cost =  0.060


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

labels = sess.run(accuracy,
                 feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1), Y: mnist.test.labels,keep_prob: 1})

fig = plt.figure()

for i in range(10):
    # 2행 5열의 그래프를 만들고, i+1 번 째에 숫자 이미지를 출력한다.
    subplot = fig.add_subplot(2, 5, i + 1)
    # 이미지를 깨끗하게 출력하기 위해 x와 y의 눈금을 출력하지 않는다.
    subplot.set_xticks([])
    subplot.set_yticks([])
    # 출력한 이미지 위에 예측한 숫자를 출력한다.
    # np.argmax는 tf.argmax와 같은 기능의 함수이다.
    # 결과값인 labels의 i번째 요소가 원-핫 인코딩 형식으로 되어 있으므로,
    # 해당 배열에서 가장 높은 값을 가진 인덱스를 예측한 숫자로 출력한다.
    subplot.set_title('%d' %np.argmax(labels[i]))
    # 1차원 배열로 되어 있는 i번째 이미지 데이터를
    # 28 * 28 형식의 2차원 배열로 변형하여 이미지 형태로 출력한다
    # cmap 파라미터를 통해 이미지를 그레이스케일로 출력한다.
    subplot.imshow(mnist.test.images[i].reshape((28, 28)),
                   cmap=plt.cm.gray_r)

# 그래프 출력
plt.show()